In [1]:
import pandas as pd
import scipy
import numpy as np
import re
import nltk
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
#ratings = pd.merge(movies, ratings).drop(['date_added', 'release_year', 'rating', 'duration'], axis=1)
ratings = pd.merge(movies, ratings)

ratings.drop('timestamp', axis = 1, inplace=True)
ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


In [3]:
ratings['rating'] = ratings['rating'].fillna('0')
ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


In [4]:
ratings['clean_genre'] = ratings['genres'].str.lower()
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: re.sub(' ', '', x))
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: re.sub('\s+', ' ', x))
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: x.split('|'))
#pd.set_option('display.max_rows', 100)
#ratings['clean_genre']
ratings

,movieId,title,genres,userId,rating,clean_genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,"[adventure, animation, children, comedy, fantasy]"
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,"[adventure, animation, children, comedy, fantasy]"
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,"[adventure, animation, children, comedy, fantasy]"
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,"[adventure, animation, children, comedy, fantasy]"
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,"[adventure, animation, children, comedy, fantasy]"
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,"[action, animation, comedy, fantasy]"
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,"[animation, comedy, fantasy]"
100833,193585,Flint (2017),Drama,184,3.5,[drama]
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,"[action, animation]"


In [5]:
genre_ratings = ratings[['title', 'userId', 'rating', 'clean_genre']].copy()
genre_ratings.head()

,title,userId,rating,clean_genre
0,Toy Story (1995),1,4.0,"[adventure, animation, children, comedy, fantasy]"
1,Toy Story (1995),5,4.0,"[adventure, animation, children, comedy, fantasy]"
2,Toy Story (1995),7,4.5,"[adventure, animation, children, comedy, fantasy]"
3,Toy Story (1995),15,2.5,"[adventure, animation, children, comedy, fantasy]"
4,Toy Story (1995),17,4.5,"[adventure, animation, children, comedy, fantasy]"


In [6]:
def get_similar_genre(genre):
    #similar_genre = genre_ratings['user_id', 'clean_genre']
    temp = []
    df = pd.DataFrame()
    #print(genre_ratings['clean_genre'][0][0])
    for i, x in enumerate(genre_ratings['clean_genre']):
        for y in x:
            if(y in genre):
                temp.append(i)
                     
    #temp = genre_ratings.loc[genre_ratings['clean_genre'][0][0] == ([genre])]
    #temp = genre_ratings.loc[genre_ratings['clean_genre'].str.contains(genre)]
    temp = set(temp)
    temp = list(temp)
    df = genre_ratings.iloc[temp]
    return df

In [7]:
df_genre = get_similar_genre(["children"])
df_genre
#df_genre.drop_duplicates(inplace=True)
#get_similar_genre(["adventure"])

,title,userId,rating,clean_genre
0,Toy Story (1995),1,4.0,"[adventure, animation, children, comedy, fantasy]"
1,Toy Story (1995),5,4.0,"[adventure, animation, children, comedy, fantasy]"
2,Toy Story (1995),7,4.5,"[adventure, animation, children, comedy, fantasy]"
3,Toy Story (1995),15,2.5,"[adventure, animation, children, comedy, fantasy]"
4,Toy Story (1995),17,4.5,"[adventure, animation, children, comedy, fantasy]"
...,...,...,...,...
65507,Snow Dogs (2002),288,3.0,"[adventure, children, comedy]"
65508,Snow Dogs (2002),380,2.0,"[adventure, children, comedy]"
6510,"Goofy Movie, A (1995)",136,1.0,"[animation, children, comedy, romance]"
32766,Hercules (1997),19,4.0,"[adventure, animation, children, comedy, musical]"


In [8]:
recc_ratings = df_genre.pivot_table(index=['userId'],columns=['title'],values='rating')
recc_ratings.fillna(0, inplace=True)
recc_ratings

title,*batteries not included (1987),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),102 Dalmatians (2000),1776 (1972),3 Ninjas (1992),3 Ninjas Kick Back (1994),3 Ninjas Knuckle Up (1995),3 Ninjas: High Noon On Mega Mountain (1998),...,Wizards of Waverly Place: The Movie (2009),Wow! A Talking Fish! (1983),"Yearling, The (1946)",Yogi Bear (2010),Yongary: Monster from the Deep (1967),Young Sherlock Holmes (1985),"Yours, Mine and Ours (1968)",Zathura (2005),Zeus and Roxanne (1997),Zootopia (2016)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
recc_columns = recc_ratings.sum(axis=0).sort_values(ascending=False)
#recc_columns = recc_ratings.pivot_table(columns=[''])
recc_columns = pd.DataFrame(recc_columns)
recc_columns['sum'] = recc_ratings.sum(axis=0).sort_values(ascending=False)
#recc_columns.(columns=recc_columns[0], axis=1, inplace=True)
del recc_columns[recc_columns.columns[0]]
recc_columns

,sum
title,
Toy Story (1995),843.0
Aladdin (1992),694.0
"Lion King, The (1994)",678.0
Shrek (2001),657.5
Finding Nemo (2003),558.5
...,...
The Star Wars Holiday Special (1978),0.5
Born to Be Wild (1995),0.5
Tooth Fairy 2 (2012),0.5


In [10]:
recc_normalization = df_genre['title'].value_counts()
recc_normalization = pd.DataFrame(recc_normalization)
recc_normalization.rename(columns={'title':'count'}, inplace=True)
recc_normalization

,count
Toy Story (1995),215
Aladdin (1992),183
"Lion King, The (1994)",172
Shrek (2001),170
Beauty and the Beast (1991),146
...,...
On the Trail of the Bremen Town Musicians (1973),1
Gena the Crocodile (1969),1
"Little Drummer Boy, The (1968)",1
Gulliver's Travels (1996),1


In [11]:
#recc_merge = pd.join(recc_columns, recc_normalization)
recc_merge = recc_columns.join(recc_normalization)
recc_merge

,sum,count
title,,
Toy Story (1995),843.0,215
Aladdin (1992),694.0,183
"Lion King, The (1994)",678.0,172
Shrek (2001),657.5,170
Finding Nemo (2003),558.5,141
...,...,...
The Star Wars Holiday Special (1978),0.5,1
Born to Be Wild (1995),0.5,1
Tooth Fairy 2 (2012),0.5,1


## Normalizing + removing count values <= 100

In [12]:
recc_merge['normalize'] = (recc_merge['sum']/recc_merge['count'])
recc_merge['count'].dtypes
#recc_merge = recc_merge.drop(recc_merge[recc_merge['count'] < 100].index, inplace=True)
recc_merge = recc_merge[(recc_merge['count'] > 100)]
recc_merge.sort_values(by='count', ascending=False).head(10)

,sum,count,normalize
title,,,
Toy Story (1995),843.0,215,3.920930
Aladdin (1992),694.0,183,3.792350
"Lion King, The (1994)",678.0,172,3.941860
Shrek (2001),657.5,170,3.867647
Beauty and the Beast (1991),550.5,146,3.770548
Finding Nemo (2003),558.5,141,3.960993
"Monsters, Inc. (2001)",511.0,132,3.871212
Babe (1995),467.5,128,3.652344
"Incredibles, The (2004)",479.5,125,3.836000


In [ ]:
ratings['cast'] = ratings['cast'].fillna(' ')
ratings['fav_cast'] = ratings['fav_cast'].fillna(' ')
ratings['director'] = ratings['director'].fillna(' ')
ratings['country'] = ratings['country'].fillna(' ')

ratings.head(61)

# User - User

### Genre

In [ ]:
ratings['clean_genre'] = ratings['fav_genre'].str.lower()
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: re.sub(' ', '', x))
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: re.sub('\s+', ' ', x))
#ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: x.split(','))
#ratings['clean_genre'] = ratings['clean_genre'].str.split(',')
pd.set_option('display.max_rows', 100)
#ratings['clean_genre']
ratings

In [ ]:
genre_ratings = ratings[['show_id', 'title', 'listed_in', 'user_id', 'user_rating', 'clean_genre']].copy()
genre_ratings.head(10)

In [ ]:
def get_similar_genre(genre):
    #similar_genre = genre_ratings['user_id', 'clean_genre']
    temp = pd.DataFrame()
    
    #temp = genre_ratings.loc[genre_ratings['clean_genre'].isin([genre])]
    temp = genre_ratings.loc[genre_ratings['clean_genre'].str.contains(genre)]
    return temp

In [ ]:
recc_data = get_similar_recc('action&adventure')
recc_data

In [ ]:
recc_ratings = recc_data.pivot_table(index=['user_id'],columns=['title'],values='user_rating')
recc_ratings.fillna(0, inplace=True)
recc_ratings

In [ ]:
recc_columns = recc_ratings.sum(axis=0).sort_values(ascending=False)
recc_columns

In [ ]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='user_rating')
print("Before: ",userRatings.shape)
#userRatings = userRatings.dropna(how='all').fillna(0)
userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)
userRatings.head(10)

In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

In [ ]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_temp = userRatings
df_std = df_temp.apply(standardize)

user_similarity = cosine_similarity(df_std.T)
user_similarity

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_dist = euclidean_distances(df_std.T)
euclidean_dist

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity,index=userRatings.columns,columns=userRatings.columns)
user_similarity_df

In [ ]:
def get_similar_cosine(movie_name, user_rating):
    similar_score = user_similarity_df[movie_name]
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [ ]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_cosine(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(10)

In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

In [ ]:
def get_similar_pearson(movie_name, user_rating):
    similar_score = corrMatrix[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [ ]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_pearson(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(10)

In [ ]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_pearson(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

#### Evalaution Metric : RMSE ( Root Mean Squared Error )

In [ ]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    return sqrt(mean_squared_error(pred, actual))

# Predict ratings on the training data with both similarity score
user_prediction = predict(euclidean_dist, corrMatrix, type='user')
item_prediction = predict(euclidean_dist, corrMatrix, type='item')
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, corrMatrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, corrMatrix)))